In [ ]:
import yfinance as yf
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd


In [ ]:
TICKER = 'SPY'
WINDOW = 3
FAST = 10
SLOW = 27

def get_data():
    df = yf.download(TICKER, period='1y')#, auto_adjust=False)
    df.columns = df.columns.get_level_values(0)
    
    return df

def add_moving_averages(df, fast, slow):
    df[f'{FAST}_ma'] = df['Close'].rolling(fast).mean()
    df[f'{SLOW}_ma'] = df['Close'].rolling(slow).mean()
    return df.dropna()

df = get_data()
df = add_moving_averages(df, FAST, SLOW)

df


In [ ]:
def add_strategy(df):
    df.columns = df.columns.get_level_values(0)
    df['Strategy'] = np.where(df['Close'] > df['MA'], 1, -1)
    df['Strategy'] = df['Strategy'].shift(1)
    return df

def test_strategy(df):
    df['Asset_Culmulative']  = np.cumprod(1+df['Close'].pct_change()) -1
    df['Strategy_Culmulative'] = np.cumprod(1+df['Close'].pct_change() * df['Strategy']) -1
    
    plt.plot(df['Asset_Culmulative'])
    plt.plot(df['Strategy_Culmulative'])
    plt.title(f'{TICKER} Close vs Moving Average')
    plt.legend([f'{TICKER} Close Price', f'{WINDOW} Moving Average'])
    
    return df

def main():
    df = get_data()
    df = add_strategy(df)
    df = test_strategy(df)
    return df

main()